In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate
from IPython.display import Image

bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

In [0]:
# ls /kaggle/input/train_images/

In [0]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [0]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

sample_submission_df = pd.read_csv("../input/sample_submission.csv")

In [0]:
train_df.head()

In [0]:
tfms = get_transforms(do_flip=False)

In [0]:
data = ImageDataBunch.from_csv(path="../input/train_images/", csv_labels="../train.csv", fn_col="file_name", label_col="category_id", ds_tfms=tfms, size=32)

In [0]:
data.show_batch(rows=3)
# data.classes

In [0]:
# train_df["height"].value_counts()

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [0]:
help(cnn_learner)

In [0]:
learn.fit_one_cycle(1)

In [0]:
learn.model_dir='/kaggle/working/'
learn.save('stage-1')

In [0]:
learn.unfreeze()

In [0]:
# learn.fit_one_cycle(1)

In [0]:
learn.load('stage-1');

In [0]:
learn.lr_find()

In [0]:
# learn.recorder.plot()

In [0]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-3))

In [0]:
img = test_df['file_name'][23678]
imgpath = '../input/test_images/' + img
# from IPython.display import Image
# Image(filename=imgpath)

In [0]:
# data_test = ImageDataBunch.from_csv(path="../input/test_images/", csv_labels="../test.csv", fn_col="file_name", ds_tfms=tfms, size=32)
# test_df

In [0]:
learn.predict(open_image(fn=imgpath))

In [0]:
data_test = ImageDataBunch.from_df(path="../input/test_images/", df=test_df, fn_col="file_name", ds_tfms=tfms, size=32)
# learn.data.add_test(data_test)
# test_df.head()

In [0]:
# learn.data.add_test(data_test) 
data_test

In [0]:
img_paths = test_df['file_name'].tolist()

In [0]:
preds = []

for i in range(0, len(img_paths)):
    file_name = img_paths[i];
    tens = learn.predict(open_image(fn='../input/test_images/'+ file_name))
    preds.append((file_name[:-4], int(tens[0]) ))
result = pd.DataFrame(preds, columns=['Id', 'Predicted'])

In [ ]:
result.to_csv('submission.csv',index=False)